In [1]:
cd g://douban_spider/

g:\douban_spider


In [2]:
import pandas as pd
import numpy as np

In [3]:
raw_rating_df = pd.read_csv('new_ratings_all.txt', sep=':', header=None, names=['uid', 'iid', 'rating', 'date', 'comment_type'], dtype={'uid':np.str, 'iid':np.str})